In [ ]:
manager.trigger_nas_processing()

In [ ]:
# duck db
import duckdb

In [ ]:
conn = duckdb.connect(database="/mnt/AI_NAS/datalake/catalog/**/*.parquet", read_only=True)

In [ ]:
result = conn.execute("""
    SELECT * FROM read_parquet('/mnt/AI_NAS/datalake/catalog/provider=*/dataset=*/task=*/variant=*/**/data.parquet', union_by_name=True)
""").fetchdf()

In [ ]:
from openai import OpenAI
import time

openai_api_key = "EMPTY"
openai_api_base = "http://localhost:9064/v1"
client = OpenAI(
    api_key=openai_api_key,
    base_url=openai_api_base,
)

In [ ]:
# print model

models = client.models.list()
for model in models.data:
    print(model.id)

In [ ]:
import base64

def base64_encode(
    image_path,
):
    with open(image_path, "rb") as image_file:
        image_bytes = image_file.read()
    return base64.b64encode(image_bytes).decode("utf-8")
image_path = "/home/kai/workspace/DeepDocs_Project/datalake/managers/test4.jpg"
base64_image = base64_encode(
    image_path,
)
    
messages = [
        {
            "role": "user",
            "content": [
                {
                    "type": "text",
                    "text": "Extract all layout elements. Reading order does not matter.",
                },
                {
                    "type": "image_url",
                    "image_url": {
                        "url": f"data:image/jpeg;base64,{base64_image}",
                    },
                },
            ],
        },
    ]

In [ ]:


start_time = time.time()
completion = client.chat.completions.create(
    model="adapter",
    messages=messages,
    temperature=0.0,
)
end_time = time.time()
elapsed_time = end_time - start_time
tokens = completion.usage.total_tokens
tokens_per_second = tokens / elapsed_time
print(f"Elapsed time: {elapsed_time:.2f} seconds")
print(f"Tokens: {tokens}")
print(f"Tokens per second: {tokens_per_second:.2f}")

In [ ]:
result = completion.choices[0].message.content

In [ ]:
target

In [ ]:
from transformers import Qwen2VLImageProcessor

In [ ]:
processor = Qwen2VLImageProcessor.from_pretrained("/mnt/AI_NAS/AI_MODEL/checkpoints/Qwen/Qwen2.5-VL-3B-Instruct")

In [ ]:
import numpy as np
image = Image.open(image_path)
inputs = processor(
    images=[image],
    return_tensors="np",
)

In [ ]:
inputs['pixel_values']

In [ ]:
from PIL import Image, ImageDraw, ImageFont
import json
image = Image.open(image_path)
draw = ImageDraw.Draw(image)
target = json.loads(result)

for item in target:
    class_ = item.get("type", "unknown")
    bbox = item.get("bbox", [0, 0, 0, 0])
    x0, y0, x1, y1 = bbox
    x0, y0, x1, y1 = int(x0), int(y0), int(x1), int(y1)
    # 
    x0 -= 14
    x1 += 14
    y0 -= 14
    y1 += 14
    draw.rectangle([x0, y0, x1, y1], outline="red", width=2)
    draw.text((x0, y0), class_, fill="red")    

In [1]:
from datalake_client import DatalakeClient
from datasets import Dataset, load_from_disk
import os
from datetime import datetime
from glob import glob
import pandas as pd
# datalake_client.py
manager = DatalakeClient(
    nas_api_url="http://192.168.20.62:8000")

2025-06-16 09:37:23,177 - datalake_client - INFO - ✅ NAS API 서버 연결 확인: http://192.168.20.62:8000


In [2]:
manager.show_nas_dashboard()


📊 NAS Data Processing Dashboard
📦 Pending: 0개
🔄 Processing: 0개
❌ Failed: 5개
🖥️ Server Status: running
⏰ Last Updated: 2025-06-16T09:37:23.858488



In [3]:
parquets = glob("/mnt/AI_NAS/datalake/catalog/provider=*/dataset=*/task=*/**/*.parquet", recursive=True)

In [ ]:
def apply_image_path(row):
    if row['image_path']:
        path = f"/mnt/AI_NAS/datalake/{row['image_path']}"
        # check path exists
        if os.path.exists(path):
            return path
        raise FileNotFoundError(f"Image path does not exist: {path}")
    return None
error_list = []
for parquet_path in parquets:
    try:
        db = Dataset.from_parquet(parquet_path)
        db = db.map(lambda x: {"image_path": apply_image_path(x)})
        # if db에 date가 없을경우
        if "date" not in db.column_names:
            db = db.map(lambda x: {"date": datetime.now().strftime("%Y-%m-%d")})
            
        sample_image_path = db[0]['image_path']
        
        provider, dataset, task, variant, *etc = parquet_path.replace("/mnt/AI_NAS/datalake/catalog/","").split("/")[:-1]
        provider = provider.replace("provider=", "")
        dataset = dataset.replace("dataset=", "")
        task = task.replace("task=", "")
        variant = variant.replace("variant=", "")
        meta = {}
        for et in etc:
            key, value = et.split("=")
            meta[key] = value
        if "table" in variant:
            meta["mod"] = "table"        

        print(f"provider: {provider}, dataset: {dataset}, task: {task}, variant: {variant}, meta: {meta}")
        temp_path = f"./temp/{provider}/{dataset}/{task}/{variant}"
        db.save_to_disk(temp_path)
        staging_dir, job_id = manager.upload_raw_data(
            data_file=temp_path,
            provider=provider,
            dataset=dataset,
            dataset_description="설명을 변경해주세요.",
            original_source="링크를 입력해주세요.",
        )
        # manager.trigger_nas_processing()
        # manager.wait_for_job_completion(job_id, timeout=1280000)
        # uploaded_db = Dataset.from_parquet(f"/mnt/AI_NAS/datalake/migrate_test/catalog/provider={provider}/dataset={dataset}/task=raw/variant=*/data.parquet")
        # uploaded_db.save_to_disk(temp_path)
        # manager.upload_task_data(
        #     data_file=temp_path,
        #     provider=provider,
        #     dataset=dataset,
        #     task=task,
        #     variant=variant,
        #     dataset_description="설명을 변경해주세요.",
        #     original_source="링크를 입력해주세요.",
        #     **meta,  # unpack meta dictionary
        # )
    except Exception as e:
        print(f"Error processing {parquet_path}: {e}")
        error_list.append({
            "parquet_path": parquet_path,
            "error": str(e)
        })
    break

provider: huggingface, dataset: funsd_plus, task: vqa, variant: base_vqa, meta: {'lang': 'en', 'src': 'real'}


Saving the dataset (0/1 shards):   0%|          | 0/1112 [00:00<?, ? examples/s]

2025-06-16 09:37:28,844 - datalake_client - INFO - 📥 Raw data 업로드 시작: huggingface/funsd_plus
2025-06-16 09:37:28,855 - datalake_client - INFO - 📂 데이터 파일 로드 중: /home/kai/workspace/DeepDocs_Project/datalake/managers/temp/huggingface/funsd_plus/vqa/base_vqa
2025-06-16 09:37:28,859 - datalake_client - INFO - ✅ datasets 폴더 로드 완료: 1112 행
2025-06-16 09:37:28,860 - datalake_client - INFO - ✅ 데이터 파일 로드 완료: ./temp/huggingface/funsd_plus/vqa/base_vqa
2025-06-16 09:37:28,860 - datalake_client - INFO - 데이터셋 컬럼: ['image_path', 'query', 'label', 'width', 'height', 'date']
2025-06-16 09:37:28,861 - datalake_client - INFO - 🔍 JSON 변환 대상 컬럼 검사 시작
2025-06-16 09:37:28,863 - datalake_client - INFO - 📄 JSON 변환 대상 컬럼 없음
2025-06-16 09:37:28,864 - datalake_client - INFO - 🔄 파일 컬럼 표준화: image_path → file_path
2025-06-16 09:37:28,866 - datalake_client - INFO - 📄 파일 분석 결과: variant=image, 이미지컬럼=[], 파일컬럼=['image_path'], 확장자={'.jpg'}


Map:   0%|          | 0/1112 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/1112 [00:00<?, ? examples/s]

2025-06-16 09:37:31,767 - datalake_client - INFO - 📦 datasets 저장 완료: /mnt/AI_NAS/datalake/migrate_test/staging/pending/funsd_plus_raw_image_8891f826_20250616093728_kai
2025-06-16 09:37:31,768 - datalake_client - INFO - ✅ Task 데이터 업로드 완료: /mnt/AI_NAS/datalake/migrate_test/staging/pending/funsd_plus_raw_image_8891f826_20250616093728_kai
2025-06-16 09:37:31,769 - datalake_client - INFO - 🔄 NAS 처리 요청 중...
2025-06-16 09:37:31,774 - datalake_client - INFO - ✅ 처리 작업 시작됨: job_20250616_093731_773
2025-06-16 09:37:31,775 - datalake_client - INFO - ⏳ 작업 완료 대기 중: None
2025-06-16 09:37:31,779 - datalake_client - WARNING - ⚠️ 작업을 찾을 수 없음: None


Error processing /mnt/AI_NAS/datalake/catalog/provider=huggingface/dataset=funsd_plus/task=vqa/variant=base_vqa/lang=en/src=real/data.parquet: 작업 상태 조회 실패: None


In [ ]:
db = Dataset.from_parquet(f"/mnt/AI_NAS/datalake/migrate_test/catalog/provider={provider}/dataset={dataset}/task=vqa/variant=*/data.parquet")

In [ ]:
db[0]

In [ ]:
uploaded_db = Dataset.from_parquet(f"/mnt/AI_NAS/datalake/migrate_test/catalog/provider={provider}/dataset={dataset}/task=raw/variant=*/data.parquet")

In [ ]:
uploaded_db

In [ ]:
manager.upload_raw_data(
    data_file="arrow경로",
    provider="provider_name",
    dataset="dataset_name",
    dataset_description="설명을 변경해주세요.",
    original_source="링크를 입력해주세요.",
)

In [ ]:
manager.upload_task_data(
    data_file=parquet_path,
    provider=provider,
    dataset=dataset,
    task=task,
    variant=variant,
    dataset_description="설명을 변경해주세요.",
    original_source="링크를 입력해주세요.",
    **meta,  # unpack meta dictionary
)

In [ ]:
db